---
This file contains the computation needed in Section 7 of the paper "Quotients of flag varieties and their birational geometry" by L. Barban, G. Occhetta and L.E. Solá Conde.

---

# Section 7: Anticanonical divisor of the Chow quotient

## Section 7.1: Geometry of the boundary divisors

--- 
We start by defining variables for the boundary divisors:

---

In [1]:
var_names = ['A0','A1','A2','A3','B0','B1','B2','B3','C01','C02','C03','C12','C13','C23','D01','D02','D03','D12','D13','D23']
vars = SR.var(' '.join(var_names))
A0, A1, A2, A3, B0, B1, B2, B3, C01, C02, C03, C12, C13, C23, D01, D02, D03, D12, D13, D23 = vars
boundary = list(vars)

VV = Matrix(boundary).transpose();

--- 
The following function returns the vector of coefficients of a given linear combination:

---


In [2]:
def coeffs(expr, basis):
    return vector([expr.coefficient(v) for v in basis])

---
We now compute the pullback to $X$ of the ample generator $H$ of $\textrm{Pic}(\mathbb P^3)$ considering the total transform of the eight fundamental planes (see Table 2)

$$
\begin{aligned}
A_2: \qquad &A_2+B_2+  C_{01} + D_{02}+D_{12}+D_{23}\\
A_3: \qquad &A_3+B_3+  C_{01}+D_{03} + D_{13}+D_{23}\\
B_0: \qquad &A_0+B_0+  C_{01}+D_{01} + D_{02}+D_{03}\\
B_1: \qquad &A_1+B_1+  C_{01}+D_{01} + D_{12}+D_{13}\\
C_{02}:\qquad  &A_1+B_2+  C_{02}+D_{02} + D_{12}+D_{13}\\
C_{12}:\qquad  &A_0+B_2+  C_{12}+D_{02} + D_{12}+D_{03}\\
C_{13}:\qquad  &A_0+B_3+  C_{13}+D_{02} + D_{03}+D_{13}\\
C_{03}:\qquad  &A_1+B_3+  C_{03} + D_{03}+D_{12}+D_{13}\\
\end{aligned}
$$

and of the quadric $C_{23}$
$$
2q^*H = A_0+A_1+B_2+B_3 + C_{01} + C_{23} +D_{01} + D_{02}+ D_{03}+D_{12} + D_{13}+D_{23}
$$
and we get eight independent relations among the boundary divisors.

---

In [3]:
#################################################################
######  Pullback of 8 special planes and a special quadric  #####
#################################################################

pull = {
    'pA2': A2 + B2 + C01 + D23 + D02 + D12,
    'pA3': A3 + B3 + C01 + D23 + D03 + D13,
    'pB0': A0 + B0 + C01 + D02 + D03 + D01,
    'pB1': A1 + B1 + C01 + D01 + D12 + D13,
    'pC02': A1 + B2 + C02 + D12 + D13 + D02,
    'pC12': A0 + B2 + C12 + D02 + D12 + D03,
    'pC13': A0 + B3 + C13 + D02 + D03 + D13,
    'pC03': A1 + B3 + C03 + D13 + D12 + D03,
    'pC23': A0 + A1 + B2 + B3 + C01 + C23 + D01 + D02 + D03 + D12 + D13 + D23
}

##################################
# Convert expressions to vectors #
##################################

pvecs = {k: coeffs(v, boundary) for k, v in pull.items()}

##################################
######  Matrix of relations  #####
##################################

MRel = matrix(8, 20, 0)
MRel[0] = pvecs['pA2'] - pvecs['pA3']    # pA2 - pA3
MRel[1] = pvecs['pA2'] - pvecs['pB0']    # pA2 - pB0
MRel[2] = pvecs['pA2'] - pvecs['pB1']    # pA2 - pB1
MRel[3] = pvecs['pA2'] - pvecs['pC02']   # pA2 - pC02
MRel[4] = pvecs['pA2'] - pvecs['pC12']   # pA2 - pC12
MRel[5] = pvecs['pA2'] - pvecs['pC13']   # pA2 - pC13
MRel[6] = pvecs['pA2'] - pvecs['pC03']   # pA2 - pC03
MRel[7] = 2 * pvecs['pA2'] - pvecs['pC23']  # 2*pA2 - pC23

#########################################################
## We check that the relation obtained are independent ##
#########################################################

MRel.rank()

8

---
We construct the matrix $Q$ giving the quotient map from the free abelian group generated by the boundary divisors to $\textrm{Pic}(X)$. In other words the columns of the matrix $Q$ can be considered as coordinates of the classes of the boundary divisors in $\textrm{Pic}(X)$.

---

In [4]:
Q=MRel.right_kernel().basis_matrix()

## Section 7.2: Intersection numbers

---
We now define the function **orbit** which computes the $W(\tau)$-orbit of expressions of the boundary divisors.
The expressions could be polynomials, without exponents, where **all the coefficients are $\pm 1$**, and **must be entered as strings**.

The function first applies the elements of $S_4$ to the symbols $0,1,2,3$, reorders indices (if $j<i$ then $ij$ is rewritten as $ji$), and transforms the output list of strings to a list of expressions in the symbolic ring.
Then the transposition $\tau$ is applied, and the final list is cleared from duplicates. To get only the $W$-orbit of an expression use the function **permute**.

---

In [5]:
import itertools
import re

###################################################
######## This function rewrites D10 as D01 ########
###################################################

def order_consecutive_digits(s):
    # Find consecutive digit pairs that are immediately next to each other
    matches = re.findall(r'(\d)(\d)', s)
    
    for a, b in matches:
        if a > b:
            s = s.replace(a + b, b + a)
    return s

##############################################################
######## Apply all the permutations in S4 to a STRING ########
##############################################################

def permute(string):
    permutations = list(itertools.permutations("0123"))
    results = set()
    for perm in permutations:
        perm_str = ''.join(perm)
        trans_table = str.maketrans("0123", perm_str)
        permuted_string = string.translate(trans_table)
        ordered_string = order_consecutive_digits(permuted_string)
        results.add(SR(ordered_string))
    return sorted(results)

#############################################
######## Apply the transposition tau ########
#############################################

def swap_labels(s):
    swaps={
        'A0':'B3','B3':'A0',
        'A3':'B0','B0':'A3',
        'A1':'B2','B2':'A1',
        'A2':'B1','B1':'A2',
        'D01':'D23','D23':'D01',
        'D02':'D13','D13':'D02',
        'C12':'C03','C03':'C12'
    }
    pattern=re.compile('|'.join(sorted(swaps,key=len,reverse=True)))
    return pattern.sub(lambda m:swaps[m.group()],s)
    
##############################################
########  Find the orbit of a string  ########
#############################################

def orbit(string):
    original = permute(string)
    swapped = permute(str(swap_labels(string)))
    return list(set(original + swapped))

---
We show here an example of how the functions orbit and permute are used.

---

In [6]:
orbit('B0+B0') ###### One should not use 2*B0

[2*A0, 2*B0, 2*A3, 2*A2, 2*A1, 2*B3, 2*B2, 2*B1]

In [7]:
permute('B0+B0')

[2*B0, 2*B3, 2*B2, 2*B1]

### Proposition 7.2 

---
In the following block we list the intersection numbers computed in the paper considering the boundary divisors $A_0$, $C_{23}$ and $D_{01}$.

---

In [8]:
####################################
###### Non meeting divisors ########
####################################

m_0 = [0] * 7
L_0 = ["A0*A1", "A0*C01", "A0*D12", "C23*C02", "C23*D12", "D01*D02", "D01*D23"]

##################################
###### Relations from D01 ########
##################################

m_D01=[0,0,0,0,1,1,1,1]
L_D01 = ["A0*A0*D01", "C01*C01*D01", "C23*C23*D01", "A0*C01*D01", "A0*B0*D01", 
         "A0*B1*D01", "A0*C23*D01", "C01*C23*D01"]

##################################
###### Relations from C23 ########
##################################

m_C23 = [0, 0, 0, 0, -1, -1, -1, 1, 1]
L_C23 = ["A0*A0*C23", "A0*A1*C23", "A0*C01*C23", "B2*C01*C23", "D01*D01*C23", 
         "D23*D23*C23", "C01*C01*C23", "A0*B2*C23", "A0*B3*C23"]

#################################
###### Relations from A0 ########
#################################

m_A0 = [-1] * 10
L_A0 = ["B0*B0*A0", "B1*B1*A0", "B2*B2*A0", "B3*B3*A0", "C13*C13*A0", "C12*C12*A0", 
        "C23*C23*A0", "D01*D01*A0", "D02*D02*A0", "D03*D03*A0"]

---
We then use the group action to compute all the intersection numbers $E \cdot F \cdot G$ except for the cases $E=F=G$.

If $E$ and $F$ do not meet then the entry $(0,\ 'E*F')$ is stored in the list **subs**.

If $E*F*G=m$ then the entry $(m,\ 'E*F*G')$ is stored in the list **subs**.

---

In [9]:
########################################################################
######  Compute other intersection numbers using the group action  #####
########################################################################

def intnum(m_list, expr_list):
    results = set()
    for m, expr in zip(m_list, expr_list):
        for e in orbit(expr):
            results.add((m, str(e)))
    return sorted(results)

subs=intnum(m_0,L_0)+intnum(m_D01,L_D01)+intnum(m_C23,L_C23)+intnum(m_A0,L_A0)

---
To compute an intersection number using the ones stored in the list subs, we use **Maxima's ratsubst function**.  
The function **inter(D,C)** computes the intersection of a divisor $D$ expressed as a linear combination of boundary divisors and a curve $C$, expressed as a linear combination of curves which are intersections of two boundary divisors.

---

In [10]:
####################################
######  Use Maxima's ratsubst  #####
####################################

def multiple_subst(substitutions, expr):
    max_expr = expr
    for new_expr, old_expr in substitutions:
        max_expr = f'ratsubst({new_expr}, {old_expr}, {max_expr})'
    return maxima(max_expr).sage()

###################################################################################
######  Function that computes the intersection of a curve C and a divisor D  #####
###################################################################################

def inter(D,C):     
    return multiple_subst(subs,(D*C).expand())

---
Example of how to use the **inter** function.

---

In [11]:
inter(A0,A0*B0+C01*C23)

-1

---
To compute the top self-intersection of the boundary divisors we can use the relations to write a cube-free formula.

---

In [12]:
##################################################################################################
######  We start by recalling the eight linear equivalent relations among boundary divisors  #####
##################################################################################################

M=MRel.rref()
M*VV

##########################################################
##### Self-intersection of the divisors A1, C01, D01 #####
##########################################################

DivA1 = A3+C12-C23-D01+D03
DivC01 = C03+C12-C23-D01+D03+D12-D23
DivD01 = -A1+A3+C12-C23+D03

inter(DivA1,A1^2),inter(DivC01,C01^2),inter(DivD01,D01^2)


(0, 1, 2)

---
We add these relations to the list subs, which now contains every possible element $E*F*G$, with $E,F$ and $G$ boundary divisors.

---

In [13]:
##########################################
###### All the self-intersections ########
##########################################

m=[0,1,2]
L=["A0*A0*A0", "C01*C01*C01", "D01*D01*D01"]
cubes=intnum(m,L)

######################################################################
###### Now the list subs contain all the intersection numbers ########
######################################################################

subs+=cubes

## Section 7.3: The anticanonical bundle of $X$

---
Now we write two possible expressions - formulae (10), (11) in the paper - of the anticanonical bundle $-K_X$.

---

In [14]:
##########################################################
######  Two expressions of the anticanonical bundle  #####
##########################################################

Exc=A0+A1+B2+B3+2*C01+D01+2*D02+2*D03+2*D12+2*D13+D23
exc=coeffs(Exc,boundary)

KX1= -(pull['pA2']+pull['pA3']+pull['pB0']+pull['pB1']) + Exc
KX2= -(pull['pC23']+pull['pA2']+pull['pA3']) + Exc
-KX1, -KX2

(A2 + A3 + B0 + B1 + 2*C01 + D01 + D23, A2 + A3 + B2 + B3 + C01 + C23 + 2*D23)

### Proposition 7.4

---
We check the bigness of $-K_X$ - we know it is nef.

---

In [15]:
inter(-KX1,KX1^2)

12

### Proposition 7.5


---
We compute the generators of the linear system of quartics in ${\mathbb P}^3$ corresponding to $|-K_X|$

---

In [16]:
############################################
##### Linear system of quartics in  P3 #####
############################################

def vero4(v):
   x=v[0]
   y=v[1]
   z=v[2]
   t=v[3]
   return [x^4,x^3*y,x^3*z,x^3*t,x^2*y^2,x^2*y*z,x^2*y*t,x^2*z^2,x^2*z*t,x^2*t^2,x*y^3,x*y^2*z,x*y^2*t,x*y*z^2,x*y*z*t,x*y*t^2,x*z^3,x*z^2*t,x*z*t^2,x*t^3,
           y^4,y^3*z,y^3*t,y^2*z^2,y^2*z*t,y^2*t^2,y*z^3,y*z^2*t,y*z*t^2,y*t^3,z^4,z^3*t,z^2*t^2,z*t^3,t^4]

In [17]:
########################################################################
##### We impose the vanishing in 21 points,                        #####
##### 12 of them depending on two parameters a,b,                  #####
##### chosen so that we have 5 points on each of the 6 base lines  #####
########################################################################

def matvero(a,b):
    v=[]
    v.append((0,0,0,1))
    v.append((0,0,a,1))
    v.append((0,0,1,1))
    v.append((0,0,b,1))
    v.append((0,0,1,0))
    v.append((0,a,0,1))
    v.append((a,a,1,1))
    v.append((a,0,1,0))
    v.append((0,1,0,1))
    v.append((a,1,a,1))
    v.append((1,1,1,1))
    v.append((b,1,b,1))
    v.append((1,0,1,0))
    v.append((0,b,0,1))
    v.append((b,b,1,1))
    v.append((b,0,1,0))
    v.append((0,1,0,0))
    v.append((a,1,0,0))
    v.append((1,1,0,0))
    v.append((b,1,0,0))
    v.append((1,0,0,0))
    m=[]
    for i in range(21):
          m.append(vero4(v[i]))
    return matrix(m)

##########################################################################################################
##### The corresponding matrix has maximal rank and a reasonable expression in the case (a,b)=(i,-i) #####
##########################################################################################################

A=matvero(I,-I)
rank(A)

21

In [18]:
###############################################################
##### We compute now the generators of the linear system ######
###############################################################

R.<x,y,z,t>=QQ[]
S=R.fraction_field()
K=matrix(S,basis(kernel(transpose(A))))
mono=transpose(matrix(S,[[x^4,x^3*y,x^3*z,x^3*t,x^2*y^2,x^2*y*z,x^2*y*t,x^2*z^2,x^2*z*t,x^2*t^2,x*y^3,x*y^2*z,x*y^2*t,x*y*z^2,x*y*z*t,x*y*t^2,x*z^3,x*z^2*t,x*z*t^2,x*t^3,
           y^4,y^3*z,y^3*t,y^2*z^2,y^2*z*t,y^2*t^2,y*z^3,y*z^2*t,y*z*t^2,y*t^3,z^4,z^3*t,z^2*t^2,z*t^3,t^4]]))
show(K*mono)

[      -y^3*z - y*z^3 + x^3*t + y*z*t^2]
[     x^2*y*z - y^3*z - y*z^3 + y*z*t^2]
[  -y^3*z + x^2*y*t - y*z^2*t + y*z*t^2]
[  -y*z^3 + x^2*z*t - y^2*z*t + y*z*t^2]
[-y^2*z*t - y*z^2*t + x^2*t^2 + y*z*t^2]
[   x*y^2*z - y^3*z - y*z^2*t + y*z*t^2]
[                      -y^3*z + x*y^2*t]
[   x*y*z^2 - y*z^3 - y^2*z*t + y*z*t^2]
[ x*y*z*t - y^2*z*t - y*z^2*t + y*z*t^2]
[                    -y^2*z*t + x*y*t^2]
[                      -y*z^3 + x*z^2*t]
[                    -y*z^2*t + x*z*t^2]
[                      -y*z*t^2 + x*t^3]
[ y^2*z^2 - y^2*z*t - y*z^2*t + y*z*t^2]

# Section 8: Birational geometry of the Chow quotient

## Section 8.1: The cone of curves of $X$

### Lemma 8.1

---
The following cells, obtained from the two expressions of the anticanonical bundle presented in Section 7.3, correspond to formulae (12) and (13) in the paper

---

In [21]:
orbit('A2 + A3 + B0 + B1 + C01 + C01 + D01 + D23')

[A2 + A3 + B0 + B1 + 2*C01 + D01 + D23,
 A1 + A2 + B0 + B3 + 2*C03 + D03 + D12,
 A0 + A1 + B2 + B3 + 2*C23 + D01 + D23,
 A0 + A2 + B1 + B3 + 2*C13 + D02 + D13,
 A1 + A3 + B0 + B2 + 2*C02 + D02 + D13,
 A0 + A3 + B1 + B2 + 2*C12 + D03 + D12]

In [22]:
orbit('A2 + A3 + B2 + B3 + C01 + C23 + D23 +D23')

[A0 + A3 + B0 + B3 + C03 + C12 + 2*D03,
 A0 + A1 + B0 + B1 + C01 + C23 + 2*D01,
 A1 + A2 + B1 + B2 + C03 + C12 + 2*D12,
 A2 + A3 + B2 + B3 + C01 + C23 + 2*D23,
 A1 + A3 + B1 + B3 + C02 + C13 + 2*D13,
 A0 + A2 + B0 + B2 + C02 + C13 + 2*D02]

### Remark 8.3

---
We list all the curves whose classes generate the cones of curves of the boundary divisors, which are extremal in all the boundary divisors containing them.

---

In [65]:
raysD=[A0*D01, B0*D01, A0*D02, B0*D02, A0*D03, B0*D03, A1*D12, B1*D12, A1*D13, B1*D13, A2*D23, B2*D23]

raysC=[C01*C23,C02*C13,C03*C12] 
### The classes C*D are not listed, since they are numerically equivalent to some A*D or B*D

raysA=[A0*B0, A0*B1, A0*B2, A0*B3, A1*B0, A1*B1, A1*B2, A1*B3, A2*B0, A2*B1,A2*B2,A2*B3, A3*B0, A3*B1, A3*B2, A3*B3]
### The classes A*C are not listed, since they are not extremal in C

allrays=raysD+raysC+raysA
smallrays= raysA+raysC

In [24]:
len(allrays)

31

### Proposition 8.4 and Remark 8.5

---
We choose bases for $N^1(X)$ and $N_1(X)$. To check that they are bases we verify that the intersection product is nondegenerate using the function **numint** to compute the intersection matrix.

---

In [25]:
#########################################################################################
######   Intersection numbers of a curve/a divisor with divisors/curves in a list  ######
#########################################################################################

def numint(x,lst): 
    ze=[]
    for i in range(len(lst)):
        n=inter(lst[i],x)
        ze.append(n)
    v0=vector([ZZ(e) for e in ze])  
    return v0

##########################################
######  Define and check the bases  ######
##########################################

basis=[A0,A1,A2,A3,B0,B1,B2,(B3-A0-A1-A2-A3-B0-B1-B2)/2,C01,C02,C03,D01]
Mbasis=Matrix(basis).transpose()
##### With this choice, the basis in N^1(X) is the canonical basis of R^12: see the matrix Q #######

raybasis=[A0*B1, A0*B2, A0*B3, A1*B2, A1*B3, A2*B3, A1*B0, A2*B0, A3*B0, B0*D01, A0*D02, C02*C13]

intmat=[]
for i in range(len(basis)):
    intmat.append(numint(basis[i],raybasis))
M=matrix(intmat)
M.rank()

12

---
Now we construct the cone $NE(X)$ generated by the classes of curves in the list allrays, and we compute the number of faces of every dimension from 1 to 12. 

---

In [26]:
####################################
########   Cone of curves    #######
####################################

ray=[]
for i in range(len(allrays)):
    ray.append(numint(allrays[i],basis)) 
NE=Cone(ray) 

PNE = NE.polyhedron()
NEfaces = PNE.f_vector()[2:]

In [27]:
NEfaces

(31, 387, 2647, 10942, 28495, 47531, 50616, 33484, 12912, 2544, 189, 1)

---
We see that $NE(X)$ has 31 extremal rays, so all the rays in the list **allrays** are extremal.  
We may also compute their anticanonical degree:

---

In [28]:
numint(-KX1,allrays)

(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)

## Section 8.2: The nef cone of $X$

---
We construct the cone $\mbox{Nef}(X)$ as the dual of $\mbox{NE}(X)$, and list the generators of its rays:

---

In [29]:
Nef=NE.dual()
Nef.rays()

M(-2, -2, -2, -1, 0, 0, -3, -4, 3,  0, -1, 3),
M( 0, -1, -1, -1, 0, 1, -2, -2, 2, -1, -1, 3),
M( 0,  0,  0,  0, 0, 1,  0,  0, 1,  0,  0, 1),
M( 0,  0,  0,  0, 1, 0,  0,  0, 1,  0,  0, 1),
M(-1, -1, -2, -1, 0, 1, -3, -3, 3,  0, -1, 4),
M(-1, -1, -1, -1, 0, 0, -1, -2, 1,  0,  0, 1),
M(-1, -1, -1, -1, 1, 0, -2, -2, 2,  0,  0, 2),
M( 0, -1,  0,  0, 1, 1, -2, -1, 3, -1,  0, 3),
M( 0,  0,  0,  0, 1, 1, -1,  0, 2,  0,  0, 2),
M( 0,  0,  0,  0, 1, 1, -1,  0, 1,  0,  0, 1),
M( 0,  0,  0,  0, 1, 1,  0,  0, 1,  0,  0, 1),
M( 0, -1,  0,  0, 1, 2, -2, -1, 3, -1,  0, 3),
M(-1, -1, -1, -1, 0, 1, -2, -2, 2,  0,  0, 2),
M(-1, -1, -1,  0, 1, 1, -2, -2, 3,  0, -1, 3),
M(-1, -1, -1, -1, 0, 0, -1, -2, 2,  0,  0, 2),
M(-1, -1, -2, -1, 0, 0, -2, -3, 2,  0, -1, 3),
M(-1, -2, -1, -1, 0, 1, -3, -3, 3, -1, -1, 3),
M( 0,  0,  0, -1, 1, 1, -2, -1, 2, -1,  0, 3),
M( 0,  0, -1, -1, 1, 2, -3, -2, 3, -1,  0, 5),
M( 0, -1,  0, -1, 1, 2, -3, -2, 4, -2,  0, 5),
M( 0, -1, -1, -1, 0, 1, -2, -2, 2, -1,  0, 3),
M( 0, -1, -1,

---
The following function, **unique_numintd**, cleans a list of divisors, removing numerically equivalent ones. Numerical equivalence is tested against the list **curves** to be provided in input (a basis of $N_1(X)$).

---

In [30]:
#########################################################
####### Clear list up to numerical equivalence    #######
#########################################################

def unique_numintd(div,curves):
    seen={}
    for d in div:
        v=tuple(numint(d,curves))
        if v not in seen:
            seen[v]=d
    return list(seen.values())

---
The function **divpos** writes, if possible, a divisor $D$, given as a linear combination of boundary divisors as a linear combination of boundary divisors *with nonnegative coefficients*.
This is done by taking the image of $D$ in the Picard group, and looking for a preimage with nonnegative coefficients in the free abelian group generated by the bboundary divisors using the function **preimage**.

---

In [31]:
#########################################################
#######   Finds a preimage of an element w        #######
#######   via the linear map defined by M         #######
#######   with nonnegative entries, if possible   #######
#########################################################

def preimage(M, w):
    from sage.numerical.mip import MIPSolverException
    p = MixedIntegerLinearProgram(maximization=False, solver="GLPK")
    x = p.new_variable(nonnegative=True)
    for i in range(M.nrows()):
        p.add_constraint(sum(M[i, j] * x[j] for j in range(M.ncols())) == w[i])
    try:
        p.solve()
        v = vector([QQ(p.get_values(x[j])) for j in range(M.ncols())])
        nonzero_entries = [a for a in v if a != 0]
        if nonzero_entries:
            g = gcd(nonzero_entries)
            v = v / g
        return v
    except MIPSolverException:
        raise ValueError("No nonnegative solution exists")

##############################################################
#######   Finds an expression of every divisor in nefrays   ##
#######   as a nonnegative combination of boundary divisors ##
##############################################################

def divpos(D):
    v=vector([D.coefficient(v) for v in boundary])
    w=Q*v
    M=matrix(preimage(Q,w))
    return (M*VV)[0,0]              

---
We write the generators of the nef cone as linear combinations of boundary divisors with non negative coefficients.

---

In [32]:
################################################
#######   Generators of the Nef cone   #########
################################################

nefrays=[divpos((matrix(ray)*Mbasis)[0,0]) for ray in Nef.rays()]

---
We check that the generators are minimal, finding for each $D$ a curve on which $D$ has degree one.

---

In [33]:
###############################################################
#######   Check that all the generators are minimal   #########
###############################################################

gen=[D for D in nefrays if 1 in numint(D,allrays)]
len(gen)

189

---
We compute the top self-intersection of the minimal generators of every ray of $\textrm{Nef}(X)$. We obtain that, among the $189$ maximal contractions of $X$, $20$ are of fiber type, and $169$ are birational contractions, with eight possible values of the top self intersection of a minimal supporting divisor $D$: 

---

In [34]:
########################################################
#######   Cubes of generators of the Nef cone   ########
########################################################

cubes=[inter(D,D^2) for D in nefrays]

In [35]:
###################################################################
#######   Possible degrees of generators of the Nef cone   ########
###################################################################

[(x,cubes.count(x)) for x in Set(cubes)]

[(0, 20),
 (1, 6),
 (2, 24),
 (4, 6),
 (5, 48),
 (14, 6),
 (16, 15),
 (18, 16),
 (22, 48)]

In [36]:
################################################
#######   List  generators of degree k   #######
################################################

def degree(k):
    indices=[i for i in range(len(cubes)) if cubes[i]==k]
    return indices,[nefrays[j] for j in indices]

In [37]:
notbig=degree(0)

### Proposition 8.6

---
Using the intersection theory rules  we see that, out of the $20$ extremal rays supporting fiber type contractions, $9$ of them are generated by divisors whose squares are numerically trivial.
Equivalently the targets of these contractions are curves; since these curves are necessarily rational, then they are projective lines.

---

In [38]:
###############################################################################
#######   Divide contractions of curves and contractions to surfaces  #########
###############################################################################

surfaces = []
P1 = []

for i in notbig[0]:
    sq = numint(nefrays[i]*nefrays[i],basis)
    if sq == 0:
        P1.append(i)
    else:
        surfaces.append(i)

In [39]:
#######################################
#######   Contractions to P1  #########
#######################################

[nefrays[i] for i in P1]

[B1 + C01 + D01,
 B0 + C01 + D01,
 B0 + C03 + D03,
 B0 + C02 + D02,
 C01 + C23 + D01 + D23,
 A1 + C23 + D01,
 A0 + C23 + D01,
 A3 + C01 + D23,
 A2 + C01 + D23]

--- 
Among the supporting divisors of these contractions, there is a $W(\tau)$-orbit containing $8$ elements, consisting of the $W$-orbits of $B_0+C_{01}+D_{01}$ and $A_0+C_{23}+D_{01}$

---

In [40]:
unique_numintd(permute('B0+C01+D01'),raybasis)

[B0 + C01 + D01, B1 + C13 + D13, B0 + C02 + D02, B3 + C03 + D03]

In [41]:
numint((B2 + C23 + D23) - (B0 + C03 + D03),raybasis), numint((B3 + C03 + D03) - (B1 + C01 + D01),raybasis), numint((B2 + C12 + D12) - (B0 + C01 + D01),raybasis)


((0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
 (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
 (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0))

In [42]:
unique_numintd(permute('A0+C23+D01'),raybasis)

[A3 + C02 + D13, A1 + C02 + D13, A1 + C03 + D12, A3 + C12 + D03]

In [43]:
numint((A3 + C02 + D13) - (A0 + C23 + D01),raybasis), numint((A0 + C12 + D03) - (A2 + C01 + D23),raybasis), numint((A2 + C13 + D02) - (A1 + C23 + D01),raybasis)

((0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
 (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
 (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0))

### Proposition 8.7

---
Examining the remaining $11$ fiber type contractions (whose image is a surface), we checked that there are three $W(\tau)$-orbits, containing, respectively, $1$, $2$ and $8$ elements. The following divisors support one contraction of each class:

- $A_0 + B_0 + D_{01} + D_{02} + D_{03}$;
- $A_2 + A_3 + C_{01} + D_{23}$;
- $B_0 + C_{01} + C_{02} + D_{01} + D_{02}$.
---

In [44]:
#############################################
#######   Contractions to surfaces  #########
#############################################

[nefrays[i] for i in surfaces]

[A0 + B0 + D01 + D02 + D03,
 B0 + C01 + C23 + D01 + D23,
 B0 + C01 + C02 + D01 + D02,
 B0 + B1 + C01 + D01,
 B1 + C01 + C23 + D01 + D23,
 B2 + C01 + C23 + D01 + D23,
 A0 + C01 + C23 + D01 + D23,
 A2 + A3 + C01 + D23,
 A1 + C01 + C23 + D01 + D23,
 A2 + C01 + C23 + D01 + D23,
 A3 + C01 + C23 + D01 + D23]

In [45]:
D1=A0 + B0 + D01 + D02 + D03
unique_numintd(orbit('A0 + B0 + D01 + D02 + D03'),raybasis)

[A0 + B0 + D01 + D02 + D03]

In [46]:
D2=A2 + A3 + C01 + D23
unique_numintd(orbit('A2 + A3 + C01 + D23'),raybasis)

[A2 + A3 + C01 + D23, B0 + B2 + C02 + D02]

In [48]:
D3=B0 + C01 + C02 + D01 + D02
unique_numintd(orbit('B0 + C01 + C02 + D01 + D02'),raybasis)

IOStream.flush timed out


[A1 + C02 + C23 + D01 + D13,
 B2 + C12 + C23 + D12 + D23,
 A0 + C12 + C13 + D02 + D03,
 A1 + C02 + C03 + D12 + D13,
 B3 + C03 + C13 + D03 + D13,
 B2 + C02 + C12 + D02 + D12,
 A1 + C03 + C23 + D01 + D12,
 B0 + C02 + C03 + D02 + D03]

---
We note that, by means of the relations in the Picard group, the three divisors $D_1,D_2,D_3$ above can be written, respectively as:

- $q^*H-C_{01}$;
- $2q^*H-(B_2+B_3+C_{01}+D_{02} + D_{03} + D_{12} + D_{13} + D_{23})$;
- $2q^*H-(A_1 + B_2 +B_3+ D_{02} + D_{03} + D_{12} + D_{13})$.

---

In [49]:
qH = A2 + B2 + C01 + D23 + D02 + D12
qH2 = A0 + A1 + B2 + B3 + C01 + C23 + D01 + D02 + D03 + D12 + D13 + D23

In [50]:
numint(qH - C01 -D1,raybasis), numint(qH2 -(B2+B3+C01+D02+D03+D12+D13+D23) -D2,raybasis),numint(qH2 -(A0+A1+B2+D02+D03+D12+D13) -D3,raybasis)

((0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
 (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
 (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0))

IOStream.flush timed out


## Section 8.3: $W$-invariant contractions

### Proposition 8.11

---
We check that the cone generated by the rays of the 8 $W(\tau)$-equivalent contractions to ${\mathbb P}^1$ contains a big divisor

---

In [51]:
P1

[2, 3, 5, 9, 23, 27, 41, 58, 115]

In [52]:
P12=P1[:4]+P1[5:]       # this is the list of contractions to P1 after eliminating the W(tau) invariant element
 
baryP1=sum([nefrays[i] for i in P12])  # we compute the baricenter of the 8 W(tau) equivalent contractions to P1 

inter(baryP1,baryP1^2)  # compute its self-intersection to check its bigness

288

---
In order to compute the nef cones of $X(1,2)$, $X(2,3)$, we consider first the 
intersection of the Nef cone of $X$ with the linear span of $5$ divisors corresponding to the contraction to $X(2)$ 
and one of the two $W$-orbits with $4$ elements in the set of contractions to ${\mathbb P}^1$.

---

In [62]:
CP12=[P1[i] for i in range(len(P1)) if i not in [5,6,7,8]]
CP23=[P1[i] for i in range(len(P1)) if i not in [0,1,2,3]]

CP12rays=[nefrays[i] for i in CP12] # two W-orbits of divisors given contractions to P1
CP23rays=[nefrays[i] for i in CP23]
 
CP12coords=[s*(numint(c,raybasis)*(M.inverse())) for c in CP12rays for s in (1,-1)]
CP23coords=[s*(numint(c,raybasis)*(M.inverse())) for c in CP23rays for s in (1,-1)]

N12=Cone(CP12coords, lattice=ToricLattice(12).dual()) 
N23=Cone(CP23coords, lattice=ToricLattice(12).dual()) 

Nef12=N12.intersection(Nef)
Nef23=N23.intersection(Nef)

--- 
Then we compute the rays of these two cones, and check that they are generating by divisors supporting the five initial contractions to ${\mathbb P}^1$, and five contractions to ${\mathbb P}^2$.

---

In [52]:
Nef12rays=[divpos((matrix(ray)*Mbasis)[0,0]) for ray in Nef12.rays()]
Nef23rays=[divpos((matrix(ray)*Mbasis)[0,0]) for ray in Nef23.rays()]

In [53]:
Nef12rays

[B1 + C01 + D01,
 C01 + C23 + D01 + D23,
 B0 + C01 + D01,
 B1 + C01 + C23 + D01 + D23,
 B0 + B1 + C01 + D01,
 B0 + C03 + D03,
 B0 + C02 + D02,
 B0 + C01 + C23 + D01 + D23,
 B0 + C01 + C02 + D01 + D02,
 B2 + C01 + C23 + D01 + D23]

In [54]:
numint(B0 + C01 + C02 + D01 + D02 - (B3 + C01 + C23 + D01 + D23),raybasis) 

(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)

In [55]:
Nef23rays

[A1 + C23 + D01,
 C01 + C23 + D01 + D23,
 A0 + C23 + D01,
 A1 + C01 + C23 + D01 + D23,
 A2 + A3 + C01 + D23,
 A2 + C01 + D23,
 A3 + C01 + D23,
 A0 + C01 + C23 + D01 + D23,
 A3 + C01 + C23 + D01 + D23,
 A2 + C01 + C23 + D01 + D23]

---
We check that the two cones define fiber type contractions, by looking at the self-intersection of their barycenters.

---

In [56]:
bary12=sum(Nef12rays)
bary23=sum(Nef23rays)
inter(bary12,bary12^2),inter(bary23,bary23^2)

(0, 0)

---
We finally check that the we cannot add any other $W$-orbit of fiber-type extremal rays to our previously 
computed cones unless we meet the set of  positive top self-intersection divisors.

---

In [74]:
surfaces

[1, 6, 8, 10, 12, 14, 82, 114, 124, 151, 158]

In [75]:
[nefrays[i] for i in surfaces]

[A0 + B0 + D01 + D02 + D03,
 B0 + C01 + C23 + D01 + D23,
 B0 + C01 + C02 + D01 + D02,
 B0 + B1 + C01 + D01,
 B1 + C01 + C23 + D01 + D23,
 B2 + C01 + C23 + D01 + D23,
 A0 + C01 + C23 + D01 + D23,
 A2 + A3 + C01 + D23,
 A1 + C01 + C23 + D01 + D23,
 A2 + C01 + C23 + D01 + D23,
 A3 + C01 + C23 + D01 + D23]

In [77]:
def DB(idxs):
    return sum(nefrays[i] for i in idxs)

DB1=DB([6,8,10,12,14, 1])
DB2=DB([6,8,10,12,14, 114])
DB3=DB([6,8,10,12,14, 82,124,151,158])

DB4=DB([82,114,124,151,158, 1])
DB5=DB([82,114,124,151,158, 10])
DB6=DB([82,114,124,151,158, 6,8,12,14])

inter(DB1,DB1^2), inter(DB2,DB2^2), inter(DB3,DB3^2), inter(DB4,DB4^2), inter(DB5,DB5^2), inter(DB6,DB6^2)

(162, 288, 1044, 162, 288, 1044)

## Section 8.4: The Effective cone

---
We introduce four new divisors, $H01$, $H02$, $H03$ and $CS$: the strict transforms of three planes and the strict transform of a Cayley cubic surface.

---

In [60]:
H01=pull['pA2']-C01-D01-D23; H01

A2 + B2 - D01 + D02 + D12

In [61]:
H02=pull['pB1']-C01-D02-D13; H02

A1 + B1 + D01 - D02 + D12

In [62]:
H03=pull['pB0']-C01-D03-D12; H03

A0 + B0 + D01 + D02 - D12

---
We check that $\{H01, H02,H03\}$ is an orbit.

---

In [63]:
unique_numintd(orbit('A2 + B2 - D01 + D02 + D12'),raybasis)

[A2 + B2 + D12 - D13 + D23,
 A0 + B0 + D02 + D03 - D23,
 A1 + B1 + D01 - D03 + D13]

In [64]:
CS=pull['pC23']+pull['pC02']-(A0+A1+C01+B2+B3+D01+D23+2*(D02+D03+D13+D12)); CS

A1 + B2 + C02 + C23 - D03

---
We check that $\{CS\}$ is an orbit.

---

In [79]:
unique_numintd(orbit('A1 + B2 + C02 + C23 - D03'),raybasis)

[A1 + B2 + C02 + C23 - D03]

---
We construct the cone $\mathcal E$ in $N^1(X)$ generated by the boundary divisors and these four divisors

---

In [81]:
####################################
#####    Coordinates in Pic(X)   ###
####################################

H01Q=Q*coeffs(H01,boundary)
H02Q=Q*coeffs(H02,boundary)
H03Q=Q*coeffs(H03,boundary)
CSQ=Q*coeffs(CS,boundary)

###################################################################################################
######  Columns of Q_eff contain the coordinates of the 24 divisors with respect to basis     #####
###################################################################################################

Q_eff = Q[:]
for v in [CSQ,H01Q,H02Q,H03Q]:
    Q_eff = Q_eff.augment(v)

#############################################################################################################
######## Cone generated by the image in Pic(X) of the boundary divisors + 4 special and its dual   ##########
#############################################################################################################

rays = [Q_eff.column(i) for i in range(Q_eff.ncols())]
E = Cone(rays=rays)

####################################
######  Dual of the cone E  ########
####################################

DE=E.dual()
dual=Cone(list(matrix(DE.rays())))

### Remark 8.12

---
We check that $\mathcal E$ has 24 extremal rays, so the classes of all the divisors that we considered generate an extremal ray. 

---

In [82]:
PDE = DE.polyhedron()
faces = PDE.f_vector()[2:]; faces

(294, 3583, 14884, 30821, 37672, 29728, 15880, 5854, 1480, 246, 24, 1)

### Theorem 8.13

#### Step 1 of the Proof of Theorem 8.13

---
Check that the numerical classes of the curves $A_i * B_j$ are a set of generators  for the linear span of the face of $NE(X)$ on which $-K_X$ is trivial. The classes $A_i * B_j$ are the first sixteen columns of the following matrix, which contains the classes generating the $K_X$ trivial face.

---

In [72]:
M=matrix([numint(c,basis) for c in smallrays]).transpose();M

[-1 -1 -1 -1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
[ 0  0  0  0 -1 -1 -1 -1  0  0  0  0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0 -1 -1 -1 -1  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0  0  0  0  0 -1 -1 -1 -1  0  0  0]
[-1  0  0  0 -1  0  0  0 -1  0  0  0 -1  0  0  0  0  0  0]
[ 0 -1  0  0  0 -1  0  0  0 -1  0  0  0 -1  0  0  0  0  0]
[ 0  0 -1  0  0  0 -1  0  0  0 -1  0  0  0 -1  0  0  0  0]
[ 1  1  1  0  1  1  1  0  1  1  1  0  1  1  1  0  0  0  0]
[ 0  0  0  0  0  0  0  0  1  1  0  0  1  1  0  0 -1  0  0]
[ 0  0  0  0  1  0  1  0  0  0  0  0  1  0  1  0  0 -1  0]
[ 0  0  0  0  1  0  0  1  1  0  0  1  0  0  0  0  0  0 -1]
[ 1  1  0  0  1  1  0  0  0  0  0  0  0  0  0  0  1  0  0]

In [73]:
M.rref()

[ 1  0  0  0  0  0  0 -1  0  0  0 -1  0 -1 -1 -1  0  0  1]
[ 0  1  0  0  0  0  0 -1  0  1  0  0  0  0  0 -1  0 -1  0]
[ 0  0  1  0  0  0  0  1  0 -1  0  0  0  1  1  1  0  1 -1]
[ 0  0  0  1  0  0  0  1  0  0  0  1  0  0  0  1  0  0  0]
[ 0  0  0  0  1  0  0  1  0 -1  0  1  0  0  1  1  1  0 -1]
[ 0  0  0  0  0  1  0  1  0  0  0  0  0  1  0  1  0  1  0]
[ 0  0  0  0  0  0  1 -1  0  1  0 -1  0 -1 -1 -2 -1 -1  1]
[ 0  0  0  0  0  0  0  0  1  1  0  0  0  0 -1 -1 -1  0  0]
[ 0  0  0  0  0  0  0  0  0  0  1  1  0  0  1  1  1  0  0]
[ 0  0  0  0  0  0  0  0  0  0  0  0  1  1  1  1  0  0  0]
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]

---
Check the numerical classes of curves in formulae (17) and (18), and their intersection numbers with the exceptional loci.

---

In [70]:
numint(A0*B0 + A1*B1 + C01*C23 + A0*D01 + B0*D01, boundary)

(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1)

In [71]:
inter(CS,A0*B0 + A1*B1 + C01*C23 + A0*D01 + B0*D01)

-1

In [72]:
numint(A0*B1 + A3*B2 + A0*C12 + B1*C12 - C03*C12, boundary)

(0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1)

In [73]:
inter(H01, A0*B1 + A3*B2 + A0*C12 + B1*C12 - C03*C12)

-1

#### Step 2 of the Proof of Theorem 8.13

In [83]:
######################################################################
########    Classes of curves contained in C_1^bir(X)     ############
######################################################################

curvesD=[A0*D01, B0*D01, A0*D02, B0*D02, A0*D03, B0*D03, A1*D12, B1*D12, A1*D13, B1*D13, A2*D23, B2*D23]
curvesC=[A0*C23, B2*C23, A2*C01, B0*C01, A1*C02, B0*C02, A0*C12, B1*C12, A1*C03, B0*C03, A0*C13,B1*C13]
curvesA1=list(orbit('A0*B1+A0*D01'))
curvesA2=list(orbit('A0*B1+A0*C12'))
curvesA3=list(orbit('A0*B0+A0*D01'))
curvesX=list(orbit('A0*B0 + A1*B1 + C01*C23 + A0*D01 + B0*D01'))
curvesY=list(orbit('A0*B1 + A3*B2 + A0*C12 + B1*C12 - C03*C12'))
divrays=curvesC+curvesD+curvesA1+curvesA2+curvesA3+curvesX+curvesY

####################################################################
########    Define a cone C contained in C_1^bir(X)     ############
####################################################################

cray=[]
for i in range(len(divrays)):
    cray.append(numint(divrays[i],basis)) 
C=Cone(cray)

###################################################
########    Check that C contains DE   ############
###################################################

all(dual.ray(i) in C for i in range(dual.nrays()))

True